In [33]:
# import autoreload
# ?autoreload

%load_ext autoreload
%autoreload 2
import torch
import time
from transformers import AutoModel
from transformers import AutoConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
from calibrate_e_ml import calibrate_e_ml
from cg.node import construct_aggregation_graph

In [25]:
seq_len = 100
bs = 4
input_ids = torch.randint(1000, size=(bs, seq_len), dtype=torch.long, device='cpu')
model_name = "prajjwal1/bert-tiny"

information = calibrate_e_ml(model_name, bs, seq_len, True)

In [26]:
input_ids.dtype

torch.int64

In [27]:
information

defaultdict(list,
            {'embedding': [{'name': 'embeddings.word_embeddings:Embedding',
               'module': Embedding(30522, 128, padding_idx=0),
               'inputs': (torch.Size([4, 100]), torch.int64),
               'outputs': (torch.Size([4, 100, 128]), torch.float32),
               'runtime': 0.00029243610333651304},
              {'name': 'embeddings.position_embeddings:Embedding',
               'module': Embedding(512, 128),
               'inputs': (torch.Size([1, 100]), torch.int64),
               'outputs': (torch.Size([1, 100, 128]), torch.float32),
               'runtime': 0.00010158796794712543},
              {'name': 'embeddings.token_type_embeddings:Embedding',
               'module': Embedding(2, 128),
               'inputs': (torch.Size([4, 100]), torch.int64),
               'outputs': (torch.Size([4, 100, 128]), torch.float32),
               'runtime': 0.0001900040078908205}],
             'layernorm': [{'name': 'embeddings.LayerNorm:LayerNorm'

In [41]:
level = information['linear'][0]
print(level)
fn = level['module']
fname=level['name']
fi_size, fi_dtype = level['inputs']
fo_size, fo_dtype = level['outputs']
fi = torch.rand(fi_size, dtype=fi_dtype)
fo = torch.rand(fo_size, dtype=fo_dtype)

fo_call = fn(fi)

{'name': 'encoder.layer.0.attention.self.query:Linear', 'module': Linear(in_features=128, out_features=128, bias=True), 'inputs': (torch.Size([4, 100, 128]), torch.float32), 'outputs': (torch.Size([4, 100, 128]), torch.float32), 'runtime': 0.0007346970960497856}


In [42]:
f_start = time.perf_counter()
for _ in range(10000):
    fo_call = fn(fi)
f_end = time.perf_counter()
print(f_end-f_start)

4.047593432012945


In [43]:
trace = torch.jit.trace(fn, fi)
trace_graph = trace.inlined_graph
trace_graph

graph(%self : __torch__.torch.nn.modules.linear.___torch_mangle_3.Linear,
      %input : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu)):
  %2 : Tensor = prim::GetAttr[name="bias"](%self)
  %3 : Tensor = prim::GetAttr[name="weight"](%self)
  %4 : Float(128:1, 128:128, requires_grad=1, device=cpu) = aten::t(%3) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %output : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::matmul(%input, %4) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1692:0
  %6 : int = prim::Constant[value=1]() # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  %m_out : Float(4:12800, 100:128, 128:1, requires_grad=0, device=cpu) = aten::add_(%output, %2, %6) # /Users/qqcao/.pyenv/versions/nlpnrg/lib/python3.8/site-packages/torch/nn/functional.py:1694:0
  return (%m_out)

In [48]:
graph, op_data_types = construct_aggregation_graph(trace_graph, fname)

/Users/qqcao/work/DetecTor/cg/op_counter.py:125: UserWarning: no counter available for prim::GetAttr!
  warnings.warn(f'no counter available for {node.op}!')
/Users/qqcao/work/DetecTor/cg/op_counter.py:125: UserWarning: no counter available for aten::t!
  warnings.warn(f'no counter available for {node.op}!')
/Users/qqcao/work/DetecTor/cg/op_counter.py:125: UserWarning: no counter available for prim::Constant!
  warnings.warn(f'no counter available for {node.op}!')


In [49]:
graph

Graph(name=encoder.layer.0.attention.self.query:Linear,
	inputs=[
		DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_3.Linear)
		DataNode(id=input, dtype=Float, shape=[4, 100, 128])],
	nodes=[
		OpNode(id=2, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_3.Linear)],
			outputs=[
				DataNode(id=2, dtype=None)],)
		OpNode(id=3, op=prim::GetAttr, 
			inputs=[
				DataNode(id=self, dtype=__torch__.torch.nn.modules.linear.___torch_mangle_3.Linear)],
			outputs=[
				DataNode(id=3, dtype=None)],)
		OpNode(id=4, op=aten::t, 
			inputs=[
				DataNode(id=3, dtype=None)],
			outputs=[
				DataNode(id=4, dtype=Float, shape=[128, 128])],)
		OpNode(id=output, op=aten::matmul, flops=13107200, mem_bytes=671744, 
			inputs=[
				DataNode(id=input, dtype=Float, shape=[4, 100, 128])
				DataNode(id=4, dtype=Float, shape=[128, 128])],
			outputs=[
				DataNode(id=output, dtype=Float, shape=[4, 100, 128])],)
		OpNode(i

In [ ]:
from datetime import datetime
print(datetime.fromtimestamp(1605498402.136237))

In [ ]:
# config = AutoConfig.from_pretrained("prajjwal1/bert-tiny")
config = AutoConfig.from_pretrained("albert-base-v2")
config.hidden_act = 'gelu_fast'
config.torchscript = True
model = AutoModel.from_config(config)
inputs = torch.randint(1000, size=(1, 100)).long()
# model()
# with torch.onnx.select_model_mode_for_export(model, False):
  # trace, _ = torch.jit._get_trace_graph(model, args=(inputs,))
#     trace = torch.jit.trace(model, (inputs, ))
mo=model.eval()